<a href="https://colab.research.google.com/github/raveesharanamukage/IMDB-Dataset-Sentiment-Analysis/blob/main/IMDB_Sentiment_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Navigate to the desired directory in your Google Drive
# %cd /content/drive/MyDrive/path/to/your/directory



Mounted at /content/drive


In [ ]:
import zipfile
zip_file_path='/content/drive/My Drive/IMDB Dataset.csv (1).zip'
output_folder_path='content/sentiment_analysis'

#Unzip the file

with zipfile.ZipFile(zip_file_path,'r') as zip_ref:
  zip_ref.extractall(output_folder_path)


In [ ]:
import pandas as pd
df=pd.read_csv('/content/content/sentiment_analysis/IMDB Dataset.csv')


In [ ]:
df.describe()


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.info() # Display column names and data types
#df.isnull().sum() # Check for missing values
#df.duplicated().sum() # Check for duplicate rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
from sklearn.model_selection import train_test_split
texts = df['review']
labels = df['sentiment']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(40000,) (40000,)
(10000,) (10000,)


## Preparing Text Data





1. **Text Standerdization**



2. **Text Splitting(tokenization)**


3. **Vocabulary Indexing**




In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import numpy as np
from sklearn.preprocessing import LabelEncoder

#Initialize TextVectorization layer
text_Vectorization=TextVectorization(max_tokens=20000,output_sequence_length=500)


#Adapt the vectorizer to the dataset(builds the vocabulary)
text_Vectorization.adapt(df['review'])

X_train_seq=text_Vectorization(X_train)
X_test_seq=text_Vectorization(X_test)

#create a LabelEncoder object
label_encoder=LabelEncoder()

#fit the encoder to labels
label_encoder.fit(y_train) #y_train contains original labels


y_train_encoded=label_encoder.transform(y_train)
y_test_encoded=label_encoder.transform(y_test)


X_train_seq_np=X_train_seq.numpy()

print(y_train_encoded[0])





0


In [ ]:
# text_Vectorization.get_vocabulary(150)  #get the vocabulary(list of tokens)

#get the token at index n
token_at_index=text_Vectorization.get_vocabulary()[124]


**Basic ANN Model**


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


def get_model(max_tokens=500,hid_dim=16):
  inputs=keras.Input(shape=(max_tokens,))
  x=layers.Dense(hid_dim,activation='relu')(inputs)
  x=layers.Dropout(0.3)(x)
  outputs=layers.Dense(1,activation='sigmoid')(x)
  model=keras.Model(inputs,outputs)
  model.compile(optimizer="rmsprop",metrics=["accuracy"],loss="binary_crossentropy")
  return model



In [ ]:
model=get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
   save_best_only=True)
 ]

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 500)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │           8,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,033 (31.38 KB)

 Trainable params: 8,033 (31.38 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train_seq,y_train_encoded,batch_size=10,epochs=10,callbacks=callbacks,validation_split=0.2)

Epoch 1/10
3200/3200 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.5015 - loss: 96.6932 - val_accuracy: 0.4997 - val_loss: 0.7728
Epoch 2/10
3200/3200 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.5012 - loss: 0.8560 - val_accuracy: 0.5009 - val_loss: 0.7214
Epoch 3/10
3200/3200 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.5022 - loss: 0.7198 - val_accuracy: 0.5008 - val_loss: 0.7084
Epoch 4/10
3200/3200 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.5009 - loss: 0.7001 - val_accuracy: 0.5001 - val_loss: 0.7085
Epoch 5/10
3200/3200 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.5000 - loss: 0.7382 - val_accuracy: 0.5001 - val_loss: 0.7072
Epoch 6/10
3200/3200 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.5005 - loss: 0.7003 - val_accuracy: 0.5000 - val_loss: 0.7064
Epoch 7/10
3200/3200 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.4987 - loss: 0.6995 - val_accuracy: 0.5005 - val_loss: 0.7050
Epoch 8/10
3200/3200 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.4912 - loss: 0.6

In [ ]:
model_instance=keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model_instance.evaluate(X_test_seq,y_test_encoded)[1]:.3f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4972 - loss: 0.7617
Test acc: 0.496


**Using LSTM model**

In [ ]:
import tensorflow as tf


inputs=keras.Input(shape=(None,),dtype="int64")

# Create a custom layer to apply tf.one_hot
class OneHotLayer(layers.Layer):
    def __init__(self, depth, **kwargs):
        super(OneHotLayer, self).__init__(**kwargs)
        self.depth = depth

    def call(self, inputs):
        return tf.one_hot(inputs, depth=self.depth)

embedded = OneHotLayer(depth=500)(inputs)
x=layers.Bidirectional(layers.LSTM(32))(embedded)
x=layers.Dropout(0.5)(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model=keras.Model(inputs,outputs)
model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])

model.summary()




Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ one_hot_layer (OneHotLayer)          │ (None, None, 500)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 64)                  │         136,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 136,513 (533.25 KB)

 Trainable params: 136,513 (533.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
   save_best_only=True)
 ]

model.fit(X_train_seq,y_train_encoded,batch_size=32,epochs=10,callbacks=callbacks,validation_split=0.2)



Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.6222 - loss: 0.6298 - val_accuracy: 0.7884 - val_loss: 0.4983
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 38s 34ms/step - accuracy: 0.8068 - loss: 0.4574 - val_accuracy: 0.8169 - val_loss: 0.4174
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 39s 33ms/step - accuracy: 0.8205 - loss: 0.4266 - val_accuracy: 0.8265 - val_loss: 0.4368
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 34s 34ms/step - accuracy: 0.8241 - loss: 0.4141 - val_accuracy: 0.8074 - val_loss: 0.4349
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.8320 - loss: 0.4048 - val_accuracy: 0.8202 - val_loss: 0.4678
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 40s 33ms/step - accuracy: 0.8342 - loss: 0.4008 - val_accuracy: 0.8260 - val_loss: 0.3980
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 41s 33ms/step - accuracy: 0.8346 - loss: 0.3982 - val_accuracy: 0.8338 - val_loss: 0.4008
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.8370 -

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define custom layer (without the @register_keras_serializable decorator)
class OneHotLayer(layers.Layer):
    def __init__(self, depth, **kwargs):
        super(OneHotLayer, self).__init__(**kwargs)
        self.depth = depth

    def call(self, inputs):
        return tf.one_hot(inputs, depth=self.depth)

    # Add a get_config method for serialization
    def get_config(self):
        config = super().get_config()
        config.update({'depth': self.depth})
        return config



# When loading the model, provide the custom_objects dictionary
model = models.load_model("one_hot_bidir_lstm.keras", custom_objects={'OneHotLayer': OneHotLayer})

print(f"Test acc: {model.evaluate(X_test_seq, y_test_encoded)[1]:.3f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.8404 - loss: 0.3751
Test acc: 0.843
